In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.1 MB/s eta 0:00:00


In [ ]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

In [ ]:
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
# PostgreSQL database connection parameters
db_params = {
    "dbname": "",
    "user": "",
    "password": "",
    "host": "",
    "port": ""
}

db = SQLDatabase.from_uri("")
db.get_usable_table_names()

# Define SQL query
query= """
SELECT * FROM smart_home_data
"""

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute(query)
        data = cur.fetchall()
        column_names = [desc[0] for desc in cur.description]  # Get column names

# Convert to Pandas DataFrame
df = pd.DataFrame(data, columns=column_names)
df

,timestamp,use,gen,dishwasher,furnace_1,furnace_2,home_office,fridge,wine_cellar,garage_door,...,apparenttemperature,pressure,windspeed,cloudcover,windbearing,precipintensity,dewpoint,precipprobability,kitchen,furnace
0,2015-12-31 21:58:58,0.631600,0.003467,0.000033,0.158883,0.063900,0.043400,0.082500,0.009083,0.012983,...,28.87,1015.98,8.20,0.75,281.0,0.0000,23.39,0.00,0.000567,0.222783
1,2015-12-31 21:59:58,1.070017,0.003500,0.000233,0.497633,0.062900,0.043583,0.170883,0.008883,0.012500,...,28.87,1015.98,8.20,0.75,281.0,0.0000,23.39,0.00,0.000583,0.560533
2,2015-12-31 22:00:58,1.016250,0.003467,0.000217,0.466683,0.062767,0.043533,0.149550,0.008900,0.012567,...,28.87,1015.98,8.20,0.75,281.0,0.0000,23.39,0.00,0.000633,0.529450
3,2015-12-31 22:01:58,0.820667,0.003467,0.000217,0.446950,0.062917,0.043583,0.010500,0.007733,0.012617,...,28.87,1015.98,8.20,0.75,281.0,0.0000,23.39,0.00,0.001083,0.509867
4,2015-12-31 22:02:58,0.539167,0.003517,0.000033,0.100133,0.116283,0.043367,0.007617,0.007083,0.013133,...,28.87,1015.98,8.20,0.75,281.0,0.0000,23.39,0.00,0.000733,0.216417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466303,2016-12-15 18:27:58,1.601233,0.003183,0.000050,0.085267,0.642417,0.041783,0.005267,0.008667,0.013483,...,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51,0.000633,0.727683
466304,2016-12-15 18:28:58,1.599333,0.003233,0.000050,0.104017,0.625033,0.041750,0.005233,0.008433,0.013433,...,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51,0.000600,0.729050
466305,2016-12-15 18:29:58,1.924267,0.003217,0.000033,0.422383,0.637733,0.042033,0.004983,0.008467,0.012933,...,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51,0.000600,1.060117
466306,2016-12-15 18:30:58,1.978200,0.003217,0.000050,0.495667,0.620367,0.042100,0.005333,0.008233,0.012817,...,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51,0.000650,1.116033


In [ ]:
llm = OpenAI(temperature=0, verbose=True, openai_api_key="")

In [ ]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

# Prompt 1

In [ ]:
df.columns

Index(['timestamp', 'use', 'gen', 'dishwasher', 'furnace_1', 'furnace_2',
       'home_office', 'fridge', 'wine_cellar', 'garage_door', 'kitchen_12',
       'kitchen_14', 'kitchen_38', 'barn', 'well', 'microwave', 'living_room',
       'temperature', 'icon', 'humidity', 'visibility', 'summary',
       'apparenttemperature', 'pressure', 'windspeed', 'cloudcover',
       'windbearing', 'precipintensity', 'dewpoint', 'precipprobability',
       'kitchen', 'furnace'],
      dtype='object')

In [ ]:
prompt1_df = df.copy()
prompt1_df['month'] = prompt1_df['timestamp'].dt.month
august_appliances = prompt1_df[prompt1_df['month'] == 8][['dishwasher', 'furnace_1', 'furnace_2', 'home_office', 'fridge', 'wine_cellar', 'garage_door', 'barn', 'well', 'microwave']]
august_appliances.sum().sort_values(ascending=False)

,0
furnace_2,5484.284317
home_office,4289.459617
wine_cellar,4260.909367
fridge,3638.700450
furnace_1,3003.453550
barn,1559.173367
dishwasher,1352.399403
well,614.176267
garage_door,595.354433
microwave,585.455817


In [ ]:
prompt1_output = agent_executor.run("In August, which appliance ('dishwasher', 'furnace_1', 'furnace_2', 'home_office', 'fridge', 'wine_cellar', 'garage_door', 'barn', 'well', 'microwave') consumed the most energy?")

<ipython-input-31-1a142e0fddd6>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  prompt1_output = agent_executor.run("In August, which appliance ('dishwasher', 'furnace_1', 'furnace_2', 'home_office', 'fridge', 'wine_cellar', 'garage_door', 'barn', 'well', 'microwave') consumed the most energy?")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:
smart_home_data I should query the schema of the smart_home_data table to see what columns are available.
Action: sql_db_schema
Action Input: smart_home_data
CREATE TABLE smart_home_data (
	timestamp TIMESTAMP WITHOUT TIME ZONE, 
	use DOUBLE PRECISION, 
	gen DOUBLE PRECISION, 
	dishwasher DOUBLE PRECISION, 
	furnace_1 DOUBLE PRECISION, 
	furnace_2 DOUBLE PRECISION, 
	home_office DOUBLE PRECISION, 
	fridge DOUBLE PRECISION, 
	wine_cellar DOUBLE PRECISION, 
	garage_door DOUBLE PRECISION, 
	kitchen_12 DOUBLE PRECISION, 
	kitchen_14 DOUBLE PRECISION, 
	kitchen_38 DOUBLE PRECISION, 
	barn DOUBLE PRECISION, 
	well DOUBLE PRECISION, 
	microwave DOUBLE PRECISION, 
	living_room DOUBLE PRECISION, 
	temperature DOUBLE PRECISION, 
	icon TEXT, 
	humidity DOUBLE PRECISION, 
	visibility DOUBLE PRECISION, 
	summary TEXT, 
	apparenttemperature DOUBLE PRECISION, 
	pressure DOUBLE PRECISION, 
	windspeed DOUBLE PRECISION

In [ ]:
prompt1_output

'furnace_2'

# Prompt 2

In [ ]:
test_output2

'The top 10 highest energy usage and fridge consumption values are [(14.71456667, 0.11625), (13.6508, 0.113566667), (13.31183333, 0.141166667), (13.03773333, 0.12325), (12.9447, 0.128516667), (12.89295, 0.1135), (12.86886667, 0.1338), (12.81918333, 0.1192), (12.78126667, 0.12085), (12.76096667, 0.0056)].'

In [ ]:
test_output2 = agent_executor.run("I am really trying to save as much money as I can. Since the fridge is consuming the most energy, can I turn it off at some times?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:
smart_home_data I should query the schema of the smart_home_data table.
Action: sql_db_schema
Action Input: smart_home_data
CREATE TABLE smart_home_data (
	timestamp TIMESTAMP WITHOUT TIME ZONE, 
	use DOUBLE PRECISION, 
	gen DOUBLE PRECISION, 
	dishwasher DOUBLE PRECISION, 
	furnace_1 DOUBLE PRECISION, 
	furnace_2 DOUBLE PRECISION, 
	home_office DOUBLE PRECISION, 
	fridge DOUBLE PRECISION, 
	wine_cellar DOUBLE PRECISION, 
	garage_door DOUBLE PRECISION, 
	kitchen_12 DOUBLE PRECISION, 
	kitchen_14 DOUBLE PRECISION, 
	kitchen_38 DOUBLE PRECISION, 
	barn DOUBLE PRECISION, 
	well DOUBLE PRECISION, 
	microwave DOUBLE PRECISION, 
	living_room DOUBLE PRECISION, 
	temperature DOUBLE PRECISION, 
	icon TEXT, 
	humidity DOUBLE PRECISION, 
	visibility DOUBLE PRECISION, 
	summary TEXT, 
	apparenttemperature DOUBLE PRECISION, 
	pressure DOUBLE PRECISION, 
	windspeed DOUBLE PRECISION, 
	cloudcover DOUBLE PRECISION, 


# Prompt 3

In [ ]:
import plotly.express as px

In [ ]:
grouped_by_month = df.copy()
grouped_by_month['month'] = grouped_by_month['timestamp'].dt.month
grouped_by_month = grouped_by_month[['month', 'dishwasher', 'furnace_1', 'furnace_2', 'home_office', 'fridge', 'wine_cellar', 'garage_door', 'barn', 'well', 'microwave']]
grouped_by_month = grouped_by_month.groupby('month').sum()
grouped_by_month

,dishwasher,furnace_1,furnace_2,home_office,fridge,wine_cellar,garage_door,barn,well,microwave
month,,,,,,,,,,
1,1681.768036,9389.889017,9918.805750,3868.911150,2357.778600,799.845933,577.553533,3133.879883,733.298401,399.164434
2,1137.593750,8097.114483,8531.244750,2546.969433,1943.472800,709.923450,502.124350,2514.573433,639.892283,289.890417
3,1379.096972,4739.744584,6926.851467,3208.097367,2123.904817,1162.985917,552.075450,1636.561550,669.413567,353.687834
4,1217.725816,3733.780467,4828.104100,2811.524267,2265.072667,1416.905917,573.291717,2420.246183,580.355833,450.248817
5,1104.000947,2025.275517,3570.331371,2587.716562,2474.500050,1504.404804,573.146583,3142.936562,387.850983,391.639317
6,1090.364864,1067.703300,2767.111100,3205.067917,2861.290717,1912.751533,598.621017,2928.489350,783.622633,330.107384
7,1109.328920,2546.349033,4590.564517,3906.742267,3315.012633,2705.757333,608.365533,2164.879650,743.712667,464.885467
8,1352.399403,3003.453550,5484.284317,4289.459617,3638.700450,4260.909367,595.354433,1559.173367,614.176267,585.455817
9,1242.334691,1269.652367,3119.498000,3393.593167,3040.410000,2598.940167,592.187200,2579.687867,566.892800,442.511600


In [ ]:
px.line(grouped_by_month)

In [ ]:
prompt3_output = agent_executor.run("What were the general energy consumption trends month by month?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: smart_home_data I should query the schema of the smart_home_data table to see what columns I can use.
Action: sql_db_schema
Action Input: smart_home_data
CREATE TABLE smart_home_data (
	timestamp TIMESTAMP WITHOUT TIME ZONE, 
	use DOUBLE PRECISION, 
	gen DOUBLE PRECISION, 
	dishwasher DOUBLE PRECISION, 
	furnace_1 DOUBLE PRECISION, 
	furnace_2 DOUBLE PRECISION, 
	home_office DOUBLE PRECISION, 
	fridge DOUBLE PRECISION, 
	wine_cellar DOUBLE PRECISION, 
	garage_door DOUBLE PRECISION, 
	kitchen_12 DOUBLE PRECISION, 
	kitchen_14 DOUBLE PRECISION, 
	kitchen_38 DOUBLE PRECISION, 
	barn DOUBLE PRECISION, 
	well DOUBLE PRECISION, 
	microwave DOUBLE PRECISION, 
	living_room DOUBLE PRECISION, 
	temperature DOUBLE PRECISION, 
	icon TEXT, 
	humidity DOUBLE PRECISION, 
	visibility DOUBLE PRECISION, 
	summary TEXT, 
	apparenttemperature DOUBLE PRECISION, 
	pressure DOUBLE PRECISION, 
	windspeed DOUBLE PRECISION, 
	

# Prompt 4

In [ ]:
appliance_df = df.copy()
appliance_df = appliance_df[['timestamp', 'dishwasher', 'furnace_1', 'furnace_2', 'home_office', 'fridge', 'wine_cellar', 'garage_door', 'barn', 'well', 'microwave']]
appliance_df['date'] = appliance_df['timestamp'].dt.date
appliance_df = appliance_df.drop(columns=['timestamp'])
appliance_df = appliance_df.groupby('date').sum()
appliance_df = appliance_df.sum(axis=1)
appliance_df.sort_values(ascending=False)

,0
date,
2016-02-13,1722.105034
2016-01-19,1483.340401
2016-02-11,1405.720573
2016-01-18,1377.647483
2016-07-30,1365.121658
...,...
2016-07-23,317.588300
2016-05-14,317.076434
2016-12-02,268.710438


In [ ]:
prompt4_output = agent_executor.run("Which day did I consume the most energy among the appliances?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: smart_home_data I should query the schema of the smart_home_data table to see what columns I can use.
Action: sql_db_schema
Action Input: smart_home_data
CREATE TABLE smart_home_data (
	timestamp TIMESTAMP WITHOUT TIME ZONE, 
	use DOUBLE PRECISION, 
	gen DOUBLE PRECISION, 
	dishwasher DOUBLE PRECISION, 
	furnace_1 DOUBLE PRECISION, 
	furnace_2 DOUBLE PRECISION, 
	home_office DOUBLE PRECISION, 
	fridge DOUBLE PRECISION, 
	wine_cellar DOUBLE PRECISION, 
	garage_door DOUBLE PRECISION, 
	kitchen_12 DOUBLE PRECISION, 
	kitchen_14 DOUBLE PRECISION, 
	kitchen_38 DOUBLE PRECISION, 
	barn DOUBLE PRECISION, 
	well DOUBLE PRECISION, 
	microwave DOUBLE PRECISION, 
	living_room DOUBLE PRECISION, 
	temperature DOUBLE PRECISION, 
	icon TEXT, 
	humidity DOUBLE PRECISION, 
	visibility DOUBLE PRECISION, 
	summary TEXT, 
	apparenttemperature DOUBLE PRECISION, 
	pressure DOUBLE PRECISION, 
	windspeed DOUBLE PRECISION, 
	